In [1]:
from nltk import tokenize
import os
from os import listdir
import random
import pickle

In [2]:
authors = ["smollett","dickens","austen","dostoyevsky","alcott","melville"]
texts = {}
for author in authors:
    files = os.listdir(f"texts/%s" % author)
    if (f"%s_extracts.pkl" % author) in files:
        files.remove(f"%s_extracts.pkl" % author)
    texts[author] = files

In [4]:
try:
    labels = []
    i = 0
    for author in texts:
        sample_sentences = []
        for text in texts[author]:
            full_text = open(f"texts/%s/%s" % (author, text), "r").read()
            full_text = full_text.replace("\n", " ")
            full_text = full_text.replace("\ufeff", "")
            sentences = tokenize.sent_tokenize(full_text)
            for sentence in sentences:
                sample_sentences += sentences
        extracts = open(f"texts/%s/%s_extracts.pkl" % (author,author), "wb")
        pickle.dump(sample_sentences,extracts)
        extracts.close()
        i += 1
except:
    extracts.close()
    print("exception occurred")
    for author in authors:
        os.remove(f"texts/%s/%s_extracts.pkl" % (author, author))

In [2]:
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras import layers
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [6]:
dickens_sentences = pickle.load(open("texts/dickens/dickens_extracts.pkl","rb"))
melville_sentences = pickle.load(open("texts/melville/melville_extracts.pkl","rb"))
austen_sentences = pickle.load(open("texts/austen/austen_extracts.pkl","rb"))
alcott_sentences = pickle.load(open("texts/alcott/alcott_extracts.pkl","rb"))
smollett_sentences = pickle.load(open("texts/smollett/smollett_extracts.pkl","rb"))
dostoyevsky_sentences = pickle.load(open("texts/dostoyevsky/dostoyevsky_extracts.pkl","rb"))

In [7]:
sentence_counts = [len(dickens_sentences), len(melville_sentences), len(austen_sentences), len(alcott_sentences), len(smollett_sentences), len(dostoyevsky_sentences)]
sentences = dickens_sentences[0:50000] + melville_sentences[0:50000] + austen_sentences[0:50000] + alcott_sentences[0:50000] + smollett_sentences[0:50000] + dostoyevsky_sentences[0:50000]

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

X_train = tokenizer.texts_to_sequences(np.array(sentences[1::5] + sentences[2::5] + sentences[3::5] + sentences[4::5]))
X_dev = tokenizer.texts_to_sequences(np.array(sentences[0::10]))
X_test = tokenizer.texts_to_sequences(np.array(sentences[5::10]))

labels = [0] * 50000 + [1] * 50000 + [2] * 50000 + [3] * 50000 + [4] * 50000 + [5] * 50000 

y_train = np.array(labels[1::5] + labels[2::5] + labels[3::5] + labels[4::5])
y_dev = np.array(labels[::10])
y_test = np.array(labels[5::10])

y_train = to_categorical(y_train, num_classes=6)
y_dev = to_categorical(y_dev, num_classes=6)
y_test = to_categorical(y_test, num_classes=6)

vocab_size = len(tokenizer.word_index) + 1

In [9]:
max_length = 50
X_train = pad_sequences(X_train, padding='post', maxlen=max_length)
X_dev = pad_sequences(X_dev, padding='post', maxlen=max_length)
X_test = pad_sequences(X_test, padding='post', maxlen=max_length)

In [10]:
emb_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=emb_dim, 
                           input_length=max_length))
model.add(layers.LSTM(100, dropout=0.1, recurrent_dropout=0.1))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(6, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            1421150   
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 16)                1616      
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 6)                

In [16]:
checkpoint_path = "cp.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

history = model.fit(X_train, y_train,
                    epochs=5,
                    verbose=True,
                    validation_data=(X_dev, y_dev),
                    callbacks=cp_callback,
                    batch_size=16)
loss, accuracy = model.evaluate(X_train, y_train, verbose=True)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=True)
print("Testing Accuracy:  {:.4f}".format(accuracy))
model_file = open("model_file.pkl", "wb")
pickle.dump(model, model_file)
model_file.close()

Epoch 1/10
15000/15000 [==============================] - ETA: 0s - loss: 0.5512 - accuracy: 0.7844
Epoch 1: saving model to cp.ckpt
15000/15000 [==============================] - 850s 56ms/step - loss: 0.5512 - accuracy: 0.7844 - val_loss: 0.5044 - val_accuracy: 0.8468
Epoch 2/10
14999/15000 [============================>.] - ETA: 0s - loss: 0.1371 - accuracy: 0.9553
Epoch 2: saving model to cp.ckpt
15000/15000 [==============================] - 706s 47ms/step - loss: 0.1371 - accuracy: 0.9553 - val_loss: 0.8514 - val_accuracy: 0.8564
Epoch 3/10
15000/15000 [==============================] - ETA: 0s - loss: 0.0799 - accuracy: 0.9724
Epoch 3: saving model to cp.ckpt
15000/15000 [==============================] - 780s 52ms/step - loss: 0.0799 - accuracy: 0.9724 - val_loss: 1.2126 - val_accuracy: 0.8548
Epoch 4/10
14999/15000 [============================>.] - ETA: 0s - loss: 0.0567 - accuracy: 0.9798
Epoch 4: saving model to cp.ckpt
15000/15000 [==============================] - 773s 52

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\esun1\.conda\envs\py_env\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\esun1\.conda\envs\py_env\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host


10169/15000 [===================>..........] - ETA: 4:01 - loss: 0.0438 - accuracy: 0.9841

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\esun1\.conda\envs\py_env\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\esun1\.conda\envs\py_env\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\esun1\.conda\envs\py_env\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\esun1\.conda\envs\py_env\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^

10170/15000 [===================>..........] - ETA: 4:06 - loss: 0.0438 - accuracy: 0.9841

...

10173/15000 [===================>..........] - ETA: 4:20 - loss: 0.0438 - accuracy: 0.9841

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\esun1\.conda\envs\py_env\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\esun1\.conda\envs\py_env\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\esun1\.conda\envs\py_env\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\esun1\.conda\envs\py_env\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^

10174/15000 [===================>..........] - ETA: 4:25 - loss: 0.0438 - accuracy: 0.9841

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\esun1\.conda\envs\py_env\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\esun1\.conda\envs\py_env\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\esun1\.conda\envs\py_env\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\esun1\.conda\envs\py_env\Lib\asyncio\selector_events.py", line 119, in _read_from_self
    data = self._ssock.recv(4096)
           ^^

10175/15000 [===================>..........] - ETA: 4:29 - loss: 0.0438 - accuracy: 0.9841

In [ ]:
plot_history(history)

NameError: name 'history' is not defined

In [12]:
checkpoint_path = "cp.ckpt"
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(X_test, y_test, verbose=2)

938/938 - 10s - loss: 1.3313 - accuracy: 0.8546 - 10s/epoch - 11ms/step
